# Imports

In [1]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import pandas as pd
from cv_helper import CvHelper
import json

# Constants
## For styling

In [2]:
colors = [(0,127,255), (127,0,255), (0,127,0), (255,0,0)]
line_padding = [0.7, 1.5,1.5,1.5]

font_scale = 1
text_position_cnt = (100, 100)
text_position_time = (100, 120)

output_folder_video = "../output/video"
output_folder_csv = "../output/csv"

## For algorithm tuning

In [3]:
video_name = "010_001_test"
video_path = f"../{video_name}.mp4"
with open(f"../output/time_coordinates/{video_name}.json", 'r') as file:
    video_metadata = json.load(file)


In [4]:
# Are for optime
kernel = np.ones((5,5),np.uint8)
threshold_area_size = video_metadata["point_thresholds"]

frame_shift = 0
set_fps = 480 # I dont know if its work



In [5]:
threshold_area_size

[100, 150, 100, 10]

In [6]:
video_path

'../010_001_test.mp4'

## Support functions

In [7]:
# def segment_marker_by_color(frame_tmp, video_color_metadata):
#     # Input must be a frame in the cielab color model from the OpenCV function
#
#     # Extract color channels
#     L_channel = frame_tmp[:, :, 0]
#     a_channel = frame_tmp[:, :, 1]
#     b_channel = frame_tmp[:, :, 2]
#
#     # Color segmentation using NumPy array operations - old one wiht pink
#     # marker_blue = (a_channel > 120) & (a_channel < 170) & (b_channel > 145)
#     # marker_blue = (a_channel > 150) & (a_channel < 170) & (b_channel > 100)
#     # marker_pink = (a_channel > 120) & (b_channel < 90) & (L_channel > 150)
#     # marker_green = (a_channel < 117) & (b_channel > 130)
#     # marker_yellow = (a_channel < 115) &  (b_channel < 115)
#
#     # Color segmentation using NumPy array operations
#     # marker_yellow =  (a_channel >  100) & (a_channel < 115) & (b_channel > -999) & (b_channel < 110) # points 0
#     # marker_pink   =  (a_channel > -999) & (a_channel < 100) & (b_channel >  130) & (b_channel < 140)  # points 1 - for human it will be light green
#     # marker_green  =  (a_channel >  100) & (a_channel < 115) & (b_channel >  140) & (b_channel < 999) # points 2 - or human it will be green
#     # marker_blue   =  (a_channel >  150) & (a_channel < 170) & (b_channel >  100) & (b_channel < 999) # points 3 - or human it will be purple
#     mask_of_points_0 = (a_channel > video_color_metadata["point_0"]["a_down"]) & \
#                     (a_channel < video_color_metadata["point_0"]["a_up"]) & \
#                     (b_channel > video_color_metadata["point_0"]["b_down"]) & \
#                     (b_channel < video_color_metadata["point_0"]["b_up"]) # points 0
#
#     mask_of_points_1 = (a_channel > video_color_metadata["point_1"]["a_down"]) & \
#                   (a_channel < video_color_metadata["point_1"]["a_up"]) & \
#                   (b_channel > video_color_metadata["point_1"]["b_down"]) & \
#                   (b_channel < video_color_metadata["point_1"]["b_up"])
#
#     mask_of_points_2 = (a_channel > video_color_metadata["point_2"]["a_down"]) & \
#                    (a_channel < video_color_metadata["point_2"]["a_up"]) & \
#                    (b_channel > video_color_metadata["point_2"]["b_down"]) & \
#                    (b_channel < video_color_metadata["point_2"]["b_up"])
#
#     mask_of_points_3 = (a_channel > video_color_metadata["point_3"]["a_down"]) & \
#                   (a_channel < video_color_metadata["point_3"]["a_up"]) & \
#                   (b_channel > video_color_metadata["point_3"]["b_down"]) & \
#                   (b_channel < video_color_metadata["point_3"]["b_up"])
#
#     # marker_yellow = (a_channel > 110) & (a_channel < 120) & (b_channel > 130) & (b_channel < 150)
#     # marker_blue =  (b_channel < 10)
#
#     kernel = np.ones((5,5),np.uint8)
#     mask_of_points_0 = cv2.dilate(np.uint8(mask_of_points_0*255),kernel,iterations = 1)
#     mask_of_points_1 = cv2.dilate(np.uint8(mask_of_points_1*255),kernel,iterations = 1)
#     mask_of_points_1 = cv2.dilate(np.uint8(mask_of_points_1*255),kernel,iterations = 1)
#     mask_of_points_2 = cv2.dilate(np.uint8(mask_of_points_2*255),kernel,iterations = 1)
#     mask_of_points_3 = cv2.dilate(np.uint8(mask_of_points_3*255),kernel,iterations = 1)
#     # marker_blue = cv2.dilate(np.uint8(marker_blue*255),kernel,iterations = 1)
#
#     return mask_of_points_0, mask_of_points_1, mask_of_points_2, mask_of_points_3


In [8]:
def frame_eval_and_draw(frame):
    """
    Process the input frame to detect markers, calculate angles, and visualize results.

    Parameters:
    - frame: Input image frame in RGB color space.
    - swap: Boolean indicating whether to swap the colors of markers.

    Returns:
    - Dictionary containing processed frame and various calculated points and angles.
    """
    # Convert the input frame to the CIELAB color space
    cielab_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2Lab)

    # Segment markers by color in the CIELAB color space
    marker_yellow, marker_pink, marker_green, marker_blue = CvHelper.segment_marker_by_color(cielab_frame, video_metadata["color_of_point"])

    # Create a stack of masks for each color marker
    masks = np.stack([marker_yellow, marker_pink, marker_green, marker_blue], axis=0)

    # Define color names for visualization
    colors_name = ["yellow", "pink", "green", "blue"]

    # Initialize a list to store points per frame
    point_per_frame = []

    # Initialize the direction vector for the first line
    direction_vector_0_1 = None

    # Iterate over each color marker
    for mask, thr, color, color_name, direction_vector in zip(
            masks, threshold_area_size, colors, colors_name, [direction_vector_0_1, None, None, None]
    ):
        # Convert the mask to uint8
        mask = np.uint8(mask)

        # Find connected components in the mask
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

        # Filter regions based on area threshold
        filtered_regions = [index for index, stat in enumerate(stats[1:]) if stat[4] >= thr]

        # Initialize a list to store points per mask
        point_per_mask = []

        # Iterate over filtered regions in the mask
        for idx, index in enumerate(filtered_regions):
            # Access region properties from the stats array
            left, top, width, height, area = stats[index + 1]

            # Calculate the centroid
            centroid_x, centroid_y = int(left + width / 2), int(top + height / 2)

            # Append the centroid to the list of points for the mask
            point_per_mask.append((centroid_x, centroid_y))

        try:
            # If direction vector is not initialized, calculate it from the first two points
            if direction_vector is None:
                direction_vector = CvHelper.calculate_vector_direction(point_per_mask[1], point_per_mask[0])
        except Exception as e:
            print(e)
            print(color_name)
            print(point_per_mask)
            print(filtered_regions)

        # Append the points for the current mask to the list of points per frame
        point_per_frame.append(point_per_mask)

    # Find the intersection point
    intersection_point_0 = CvHelper.find_intersection(point_per_frame[0][0], point_per_frame[0][1], point_per_frame[1][0], point_per_frame[1][1])
    intersection_point_1 = CvHelper.find_intersection(point_per_frame[1][0], point_per_frame[1][1], point_per_frame[2][0], point_per_frame[2][1])
    intersection_point_2 = CvHelper.find_intersection(point_per_frame[2][0], point_per_frame[2][1], point_per_frame[3][0], point_per_frame[3][1])

    # Here is calculated that tip point must be selected as intersections point
    if CvHelper.calculate_euclidean_distance(point_per_frame[3][0], point_per_frame[0][0]) < CvHelper.calculate_euclidean_distance(point_per_frame[3][0], point_per_frame[0][1]):
        trajectory_point_0_b = point_per_frame[0][0]
        trajectory_point_0_a = point_per_frame[0][1]
    else:
        trajectory_point_0_b = point_per_frame[0][1]
        trajectory_point_0_a = point_per_frame[0][0]

    if CvHelper.calculate_euclidean_distance(point_per_frame[0][0], point_per_frame[1][0]) > CvHelper.calculate_euclidean_distance(point_per_frame[0][0], point_per_frame[1][1]):
        trajectory_point_1_b = point_per_frame[1][0]
        trajectory_point_1_a = point_per_frame[1][1]
    else:
        trajectory_point_1_b = point_per_frame[1][1]
        trajectory_point_1_a = point_per_frame[1][0]

    if CvHelper.calculate_euclidean_distance(point_per_frame[2][0], point_per_frame[3][0]) > CvHelper.calculate_euclidean_distance(point_per_frame[2][0], point_per_frame[3][1]):
        trajectory_point_3_b = point_per_frame[3][0]
        trajectory_point_3_a = point_per_frame[3][1]
    else:
        trajectory_point_3_b = point_per_frame[3][1]
        trajectory_point_3_a = point_per_frame[3][0]

    if CvHelper.calculate_euclidean_distance(point_per_frame[1][0], point_per_frame[2][0]) > CvHelper.calculate_euclidean_distance(point_per_frame[1][0], point_per_frame[2][1]):
        trajectory_point_2_b = point_per_frame[2][0]
        trajectory_point_2_a = point_per_frame[2][1]
    else:
        trajectory_point_2_b = point_per_frame[2][1]
        trajectory_point_2_a = point_per_frame[2][0]


    # Calculate angles between consecutive lines
    angle_0 = CvHelper.calculate_angle(point_per_frame[0], point_per_frame[1])
    angle_1 = CvHelper.calculate_angle(point_per_frame[1], point_per_frame[2])
    angle_2 = CvHelper.calculate_angle(point_per_frame[2], point_per_frame[3])

    if np.isnan(angle_0):
        angle_0 = -1
    if np.isnan(angle_1):
        angle_1 = -1
    if np.isnan(angle_2):
        angle_2 = -1

    # Return the processed frame and calculated values
    return {
        "frame": frame,
        "angle_0": angle_0,
        "angle_1": angle_1,
        "angle_2": angle_2,
        "trajectory_point_0_a": trajectory_point_0_a,
        "trajectory_point_0_b": trajectory_point_0_b,
        "trajectory_point_1_a": trajectory_point_1_a,
        "trajectory_point_1_b": trajectory_point_1_b,
        "trajectory_point_2_a": trajectory_point_2_a,
        "trajectory_point_2_b": trajectory_point_2_b,
        "trajectory_point_3_a": trajectory_point_3_a,
        "trajectory_point_3_b": trajectory_point_3_b,
        "intersection_point_0": intersection_point_0,
        "intersection_point_1": intersection_point_1,
        "intersection_point_2": intersection_point_2,
    }


# Main logic

In [10]:
import cv2
import time
lookup_shape = (720,1080)
frameref_ms = int(time.time()*1000)
frametime_ms = int(1000/29.98)
# Add the necessary functions for main_function and add_text_to_frame
frame_shift = 0
measure = []  # for storing angles
frames_to_store = []
cnt = frame_shift  # for storing frame count
paused = False
# Initialize variables
frame_count = 30
alpha = 0.01  # Adjust this value for the ghosting effect
accumulated_frame = None
line = False
ghost = False
line_windows_1 = True
point_windows_1 = True


cap = cv2.VideoCapture(video_path)

cap.set(cv2.CAP_PROP_POS_FRAMES, frame_shift)
# cap.set(cv2.CAP_PROP_FPS, 29.98)

if not cap.isOpened():
    print("Error: Could not open the video file.")
    exit()

# Create a window to display the frames
cv2.namedWindow('Video Preview', cv2.WINDOW_NORMAL)




while True:
    frameref_ms += frametime_ms
    # print(cnt)
    strt = time.time()

    if not paused:
        ret, frame = cap.read()
        if not ret:
            print("ERRUR")
            print(cap.get(cv2.CAP_PROP_POS_FRAMES))
            break

        bckp_frame = frame.copy()

        # Abalyze frame and append data
        frame_data_json = frame_eval_and_draw(frame)
        frame_data_json_without_frame = frame_data_json.copy()
        frame_data_json_without_frame.pop("frame")
        frame_data_json_without_frame["frame"] = cnt
        measure.append(frame_data_json_without_frame)


# Add text to the frame
        frame = CvHelper.add_text_to_frame(frame, str(cap.get(1)), position=text_position_cnt, font_scale=font_scale)

        if line_windows_1:
            point0, point1 = CvHelper.line_padding(frame_data_json["trajectory_point_0_a"],frame_data_json["trajectory_point_0_b"],2)
            frame = cv2.line(frame, point0, point1, colors[0], 3)
            point0, point1 = CvHelper.line_padding(frame_data_json["trajectory_point_1_a"],frame_data_json["trajectory_point_1_b"],3)
            frame = cv2.line(frame, point0, point1, colors[1], 3)
            point0, point1 = CvHelper.line_padding(frame_data_json["trajectory_point_2_a"],frame_data_json["trajectory_point_2_b"],3)
            frame = cv2.line(frame, point0, point1, colors[2], 3)
            point0, point1 = CvHelper.line_padding(frame_data_json["trajectory_point_3_a"],frame_data_json["trajectory_point_3_b"],3)
            frame = cv2.line(frame, point0, point1, colors[3], 3)

        point_size = 7
        if point_windows_1:
            frame = cv2.circle(frame, frame_data_json["trajectory_point_0_a"], point_size, colors[0], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_0_b"], point_size, colors[0], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_1_a"], point_size, colors[1], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_1_b"], point_size, colors[1], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_2_a"], point_size, colors[2], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_2_b"], point_size, colors[2], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_3_a"], point_size, colors[3], -1)
            frame = cv2.circle(frame, frame_data_json["trajectory_point_3_b"], point_size, colors[3], -1)
            # Draw the intersection point
            # cv2.circle(frame, intersection_point_0, 2, (255, 255, 255), -1)
            # cv2.circle(frame, intersection_point_1, 2, (255, 255, 255), -1)
            # cv2.circle(frame, intersection_point_2, 2, (255, 255, 255), -1)





        rotated_points = CvHelper.rotate_frame_tracked_points(frame_data_json["trajectory_point_0_a"],
                                                              frame_data_json["trajectory_point_0_b"],
                                                              frame_data_json["trajectory_point_1_b"],
                                                              frame_data_json["trajectory_point_2_b"],
                                                              frame_data_json["trajectory_point_3_b"])




        frame2 = np.zeros((lookup_shape[0],lookup_shape[1],3))
        rotated_points[:,0] = rotated_points[:,0] + int(lookup_shape[1] / 2)
        rotated_points[:,1] = rotated_points[:,1] + int(lookup_shape[0] / 2)


        rotated_points = rotated_points.astype(int)


        cv2.circle(frame2, rotated_points[0], 5, colors[0], -1)
        cv2.circle(frame2, rotated_points[1], 5, colors[1], -1)
        cv2.circle(frame2, rotated_points[2], 5, colors[2], -1)
        cv2.circle(frame2, rotated_points[3], 5, colors[3], -1)

        # Create a copy of the frame to modify for ghosting

        ghost_frame = frame2.copy()

        # Draw the circles on the ghost frame
        cv2.circle(ghost_frame, rotated_points[0], 3, colors[0], -1)
        cv2.circle(ghost_frame, rotated_points[1], 3, colors[1], -1)
        cv2.circle(ghost_frame, rotated_points[2], 3, colors[2], -1)
        cv2.circle(ghost_frame, rotated_points[3], 3, colors[3], -1)

        # AGNLE PRINTING
        frame = CvHelper.add_text_to_frame(frame, "ANGLE 0: {}".format(int(frame_data_json["angle_0"])), position=(900, 400), font_scale=0.5, thickness=2, color=(255, 255, 0))
        frame = CvHelper.add_text_to_frame(frame, "ANGLE 1: {}".format(int(frame_data_json["angle_1"])), position=(900, 420), font_scale=0.5, thickness=2, color=(255, 255, 0))
        frame = CvHelper.add_text_to_frame(frame, "ANGLE 2: {}".format(int(frame_data_json["angle_2"])), position=(900, 440), font_scale=0.5, thickness=2, color=(255, 255, 0))


        # Accumulate the current frame with the previous frames
        if accumulated_frame is None:
            accumulated_frame = ghost_frame
        else:
            accumulated_frame = cv2.addWeighted(accumulated_frame, 1 - alpha, ghost_frame, alpha, 0)

        # Visualize the line connecting the two points
        if ghost:
            final_frame = accumulated_frame.copy()
        else:
            final_frame = frame2.copy()
        if line:
            cv2.line(final_frame, rotated_points[0], rotated_points[1], (255,255,255), 3)
            cv2.line(final_frame, rotated_points[1], rotated_points[2], (255,255,255), 3)
            cv2.line(final_frame, rotated_points[2], rotated_points[3], (255,255,255), 3)
    # Draw the joint point
        # Calculate and add time information
        end = time.time()
        frame = CvHelper.add_text_to_frame(frame, str(end - strt), position=text_position_time, font_scale=font_scale)

        cv2.imshow('Video Preview', frame)
        # cv2.imshow('Tracked point',np.resize(frame2,(int(lookup_shape[0] / 4),int(lookup_shape[1] / 4))))
        cv2.imshow('Tracked point',final_frame)
        # frames_to_store.append(frame.copy())
        cnt += 1

    # key = cv2.waitKeyEx(frameref_ms-int(time.time()*1000))  & 0xFF # Use cv2.waitKeyEx to capture arrow key presses
    key = cv2.waitKeyEx(1)  # Use cv2.waitKeyEx to capture arrow key presses

    if key == 27:  # Press 'Esc' to exit
        break
    elif key == 32:  # Press 'Space' to pause/unpause
        paused = not paused
    elif key == 103:  # Press 'Space' to pause/unpause
        print("press G")
        if ghost:
            ghost = False
        else:
            ghost = True
    elif key == 108:  # Press 'Space' to pause/unpause
        print("press L")
        if line:
            line = False
        else:
            line = True
    elif key == 112:  # Press 'Space' to pause/unpause
        print("press P")
        if point_windows_1:
            point_windows_1 = False
        else:
            point_windows_1 = True
    elif key == 107:  # Press 'Space' to pause/unpause
        print("press K")
        if line_windows_1:
            line_windows_1 = False
        else:
            line_windows_1 = True
    elif key == 97:  # Press left arrow key to move backward
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, cnt - 2))
        cnt -= 2
    elif key == 100:  # Press right arrow key to move forward
        cap.set(cv2.CAP_PROP_POS_FRAMES, cnt)
    elif key == 65364:  # Press down arrow key to move forward faster
        cap.set(cv2.CAP_PROP_POS_FRAMES, min(cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1, cnt + 30))
        cnt += 30

cap.release()
cv2.destroyAllWindows()


[ WARN:0@1.938] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@1.938] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@1.938] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@1.938] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_16645483

KeyboardInterrupt: 

In [ ]:
# plt.figure(figsize=(20,20))
# plt.imshow(cv2.cvtColor(bckp_frame, cv2.COLOR_BGR2RGB))

In [ ]:
# plt.imshow(a)

In [ ]:
# plt.imshow(b)

In [ ]:
# plt.imshow(L)
# plt.imshow(bckp_frame* np.stack([marker_yellow,marker_yellow,marker_yellow] ,axis=2))

## Store processed video

In [ ]:
# # Store the video with updated frames
# output_video_path = os.path.join(output_folder_video,f"{video_name}.mp4")  # Set the desired output video path
# store_video(frames_to_store, output_video_path, set_fps)

## Store csv - raw_angles

In [55]:
measure

[{'angle_0': 178.89454326098996,
  'angle_1': 119.0165843438658,
  'angle_2': 161.91101781210654,
  'trajectory_point_0_a': (328, 645),
  'trajectory_point_0_b': (431, 502),
  'trajectory_point_1_a': (1228, 68),
  'trajectory_point_1_b': (1246, 44),
  'trajectory_point_2_a': (673, 374),
  'trajectory_point_2_b': (644, 370),
  'trajectory_point_3_a': (725, 391),
  'trajectory_point_3_b': (762, 409),
  'intersection_point_0': (-10995, 16366),
  'intersection_point_1': (967, 414),
  'intersection_point_2': (696, 377),
  'frame': 0},
 {'angle_0': 220.24904691637872,
  'angle_1': 77.66208068847715,
  'angle_2': 163.17645013164116,
  'trajectory_point_0_a': (328, 645),
  'trajectory_point_0_b': (431, 502),
  'trajectory_point_1_a': (1228, 68),
  'trajectory_point_1_b': (1224, 17),
  'trajectory_point_2_a': (673, 374),
  'trajectory_point_2_b': (644, 370),
  'trajectory_point_3_a': (725, 391),
  'trajectory_point_3_b': (762, 408),
  'intersection_point_0': (1180, -538),
  'intersection_point_

In [12]:
df_angle = pd.DataFrame(measure)
df_angle["time"] = df_angle["frame"] / set_fps
# df_angle

In [11]:
# UNCOMMENT FOR STORE
# df_angle.to_csv(os.path.join(output_folder_csv,f"{video_name}.csv"), index=False)


In [ ]:
dd